**DVAMI20h**

- Arlind Iseni
- Alexander Jamal

## Assignment 2
The aim of Assignment 2 is to experimentally compare the computational and predictive performance of three learning algorithms on a spam detection task.

**Group assignment:** Max 2 students

**Prerequisite reading:** sections 12.1 - 12.3 in the main literature

**Language:** Python (Already implemented supervised learning algorithms and standard libraries can be used. However, It is NOT permitted to use any library or API that directly computes the Friedman and Nemeyi tests.)

**Data:** Spambase Dataset, https://archive.ics.uci.edu/ml/datasets/SpambaseLinks to an external site.

**Algorithms**  
three supervised classification learning algorithms of your choice.

**Evaluation measures:** perform a comparison between the selected algorithms based on:
1) computational performance in terms of training time, 
2) predictive performance based on accuracy. 
3) predictive performance based on F-measure.

**Procedure**  
(repeat steps 2, 3, and 4 for each evaluation measure above)

1. Run stratified ten-fold cross-validation tests.
2. Present the results exactly as in the table in example 12.4 of the main literature.
3. Conduct the ***Friedman test*** and report the results exactly as in the table in example 12.8 of the main literature.
4. Determine whether the average ranks as a whole display significant differences on the **0.05** $\alpha$-level and, if so, use the Nemeyi test to calculate the critical difference in order to determine which algorithms perform significantly different from each other.

**Compute**  
the size of possible instances
the size of hypothesis space (the number of possible extensions)
the number of possible conjunctive concepts according to the descriptions in Section 4.1 of the main literature
Implement the algorithm and verify that it works as expected.
Compute the accuracy of the model and report the generated model, i.e., the conjunctive rule.

**Written report**  
Template: The IEEE conference template and citation style should be followed (templatesLinks to an external site. in MS word and LaTeX).
Language: English without spelling mistakes.
Style: Clear.
Content: The report should give an overview of the conducted experiments and the obtained results. It should contain (but not be limited to) information about the used classifiers, a brief description of the Friedman and Nemeyi tests along with the formulas, results of the experiment as stated above, results of the comparison stating whether the algorithms perform significantly different or not from each other for each performance measure.
Format: PDF.
Page limit: 2 pages excluding references (no abstract should be included)

**Code**   
Provide meaningful comments for different blocks of the code. 
A README.TXT file must clearly state exactly how to execute the code and any necessary setups.

**Submission**  
Make sure to include your names in the report and the code.
The report must be submitted as a PDF separately (not to be included in the ZIP file).
Code and additional files related to implementation must be archived using ZIP.

### Import modules and dataset

In [1]:
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import f1_score, accuracy_score
from scipy.stats import friedmanchisquare
# import custom module for friedman table
from friedman_table import Friedman

### Load and read dataframe

In [2]:
# columns are saved in the data/names.txt file. all entries without the newline character in a list.
with open("data/names.txt", "r") as f:
    columns = f.read().splitlines()

In [3]:
df = pd.read_csv("data/spambase.data", names=columns)

In [4]:
df.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_orders,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


### Data exploration

In [5]:
# percentage of spam and non spam in data set
df["is_spam"].value_counts(normalize=True)

0    0.605955
1    0.394045
Name: is_spam, dtype: float64

### Data cleaning

In [6]:
# if any missing values
df.isna().sum().any()

False

In [7]:
# if there exists duplicates
df.duplicated().sum()

391

In [8]:
# if there exists any values below zero (errors)
(df < 0).all().any()

False

### Data transformation

In [9]:
def fit(X_: pd.DataFrame, params: dict) -> KBinsDiscretizer:
    """
    Description:
        Fits an algorithm to data.
    
    Args:
        X_: pandas dataframe (unlabeled data).
        params: parameters used to initialize transformer function.
    
    Returns:
        Returns transformation function object.
    """
    return KBinsDiscretizer(**params).fit(X_)

In [10]:
def discretizer(X_: pd.DataFrame, fitter_: KBinsDiscretizer) -> pd.DataFrame:
    """
    Description:
        Transforms dataframe.

    Args:
        X_: pandas data frame (unlabeled data).
        fitter_: KBinsDiscretizer object we use to transform our data.

    Returns:
        Returns pandas dataframe.
    """
    return pd.DataFrame(fitter_.transform(X_))

### Data spliting

In [11]:
# split data into target (label, Y) and non-target (X)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [12]:
def train_test(X_: pd.DataFrame, y_: pd.Series, clf, metric, params={}, *, timer=False) -> list[float] or float:
    """
    Description:
        Loops through k folds (balanced) and trains and tests classifier.

    Args:
        X_: pandas data frame (unlabeled data).
        y_: pandas series (label data).
        clf: machine learning classifier.
        metric: performance function.
        params: dictionary with arguments for the transformer.

    Returns:
        List of values for a given metric.
    """
    # split our data into 10 folds
    skf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
    # empty lists for appedning values
    metric_lst = []
    time_lst = []

    for train_index, test_index in skf.split(X_, y_):

        # if we dont have any parameters we set fitter to None so we dont fit and transform data
        fitter = None if params == {} else fit(X_.iloc[train_index], params)
        # split X into train and test. if we fit and transform, discretize X
        X_train = X_.iloc[train_index] if fitter == None else discretizer(X_.iloc[train_index], fitter)
        X_test = X_.iloc[test_index] if fitter == None else discretizer(X_.iloc[test_index], fitter)
        y_train, y_test = y_[train_index], y_[test_index]

        # start time
        st = time()
        # fit data
        clf.fit(X_train, y_train)
        # end time
        et = time()
        # predict data
        y_pred = clf.predict(X_test)
        
        # append time and metric
        time_lst.append(et - st)
        metric_lst.append(metric(y_test, y_pred))

    return metric_lst if timer == False else time_lst

### Experiment

https://medium.com/mlearning-ai/comparing-classifiers-friedman-and-nemenyi-tests-32294103ee12

In [13]:
# instantiation
svm = SVC()
ada = AdaBoostClassifier()
rf = RandomForestClassifier()

In [14]:
# number of samples
blocks = 10

In [15]:
# treatment groups (accuracy)
treatments_accuracy = {
    "Support Vector Machine": train_test(X, y, svm, accuracy_score, params=dict(n_bins=7, encode="ordinal", strategy="kmeans")), 
    "AdaBoost": train_test(X, y, ada, accuracy_score),
    "Random Forest": train_test(X, y, rf, accuracy_score)}

In [16]:
# treatment groups (f1-measure)
treatments_f1 = {
    "Support Vector Machine": train_test(X, y, svm, f1_score, params=dict(n_bins=7, encode="ordinal", strategy="kmeans")), 
    "AdaBoost": train_test(X, y, ada, f1_score),
    "Random Forest": train_test(X, y, rf, f1_score)}                          

In [17]:
# treatment groups (time)
treatments_time = {
    "Support Vector Machine": train_test(X, y, svm, f1_score, params=dict(n_bins=7, encode="ordinal", strategy="kmeans"), timer=True), 
    "AdaBoost": train_test(X, y, ada, f1_score, timer=True),
    "Random Forest": train_test(X, y, rf, f1_score, timer=True)}

In [18]:
accuracy = Friedman(blocks, treatments_accuracy)
f1 = Friedman(blocks, treatments_f1)
time = Friedman(blocks, treatments_time)

In [19]:
accuracy.create_table()
f1.create_table()
time.create_table(asc=True)

In [21]:
accuracy.get_table

,Support Vector Machine,AdaBoost,Random Forest
1,0.9240780911062907 (3),0.9392624728850325 (2),0.9436008676789588 (1)
2,0.9326086956521739 (3),0.95 (2),0.95 (2)
3,0.9282608695652174 (3),0.9369565217391305 (2),0.9369565217391305 (2)
4,0.9434782608695652 (2),0.9369565217391305 (3),0.9478260869565217 (1)
5,0.9434782608695652 (3),0.9565217391304348 (2),0.9565217391304348 (2)
6,0.9456521739130435 (3),0.9478260869565217 (2),0.9586956521739131 (1)
7,0.95 (2),0.9456521739130435 (3),0.9695652173913043 (1)
8,0.9456521739130435 (3),0.9652173913043478 (2),0.9717391304347827 (1)
9,0.8934782608695652 (2),0.8456521739130435 (3),0.9 (1)
10,0.8760869565217392 (1),0.8565217391304348 (3),0.8565217391304348 (3)


In [22]:
f1.get_table

,Support Vector Machine,AdaBoost,Random Forest
1,0.9014084507042254 (3),0.9199999999999999 (2),0.9382022471910112 (1)
2,0.9131652661064426 (3),0.9362880886426593 (2),0.9385474860335197 (1)
3,0.9065155807365438 (3),0.9169054441260746 (1),0.9069767441860465 (2)
4,0.9257142857142856 (2),0.9196675900277009 (3),0.9398280802292263 (1)
5,0.9265536723163841 (3),0.943502824858757 (1),0.9408450704225353 (2)
6,0.9322493224932249 (3),0.9361702127659575 (2),0.9398907103825136 (1)
7,0.9329446064139941 (2),0.9283667621776505 (3),0.9604519774011299 (1)
8,0.9283667621776505 (3),0.956043956043956 (2),0.9664804469273743 (1)
9,0.8650137741046833 (2),0.8202531645569621 (3),0.8720626631853786 (1)
10,0.835734870317003 (1),0.8156424581005587 (3),0.8199445983379502 (2)


In [23]:
time.get_table

,Support Vector Machine,AdaBoost,Random Forest
1,0.3562939167022705 (2),0.3331410884857178 (1),0.574091911315918 (3)
2,0.3374021053314209 (2),0.33661890029907227 (1),0.5636858940124512 (3)
3,0.34873318672180176 (2),0.33718109130859375 (1),0.5550539493560791 (3)
4,0.34514379501342773 (2),0.33376216888427734 (1),0.5718448162078857 (3)
5,0.3582429885864258 (2),0.33681583404541016 (1),0.5613400936126709 (3)
6,0.3553600311279297 (2),0.3364989757537842 (1),0.5745770931243896 (3)
7,0.36253905296325684 (2),0.33905529975891113 (1),0.5780630111694336 (3)
8,0.35586094856262207 (2),0.3428468704223633 (1),0.561147928237915 (3)
9,0.3271920680999756 (1),0.3365020751953125 (2),0.5934309959411621 (3)
10,0.33400917053222656 (1),0.3396339416503906 (2),0.5883619785308838 (3)


In [31]:
accuracy.friedman_statistic(), f1.friedman_statistic(), time.friedman_statistic()

(7.333333333333333, 7.800000000000001, 16.8)

Only the time table rejected the null-hypotheses. Now we continue with the nemenyi test.

### Nemenyi test

In order to conduct the ***Nemenyi*** test on the 3 algorithms we have, we will be calculating the mean value of the rankings of each algorithm which has been done in *f1*.

We continue by calculating the critical distance (CD) using the following formula: CD = q\$_\alpha \times \sqrt{\frac{k \times (k+1)}{6n}}$

where $q_\alpha$ depends on the significance level $\alpha$ as well as **k**: for $\alpha$ = 0.05 and **k** = 3 it is X, **n** is the amount of measurements taken which in our case is **n** = 10, which leads to a **CD** of X. Since our average ranks for Support Vector Machine, AdaBoost and Random Forest are *X1, X2 and X3* respectively, ...

In [ ]:
time.critical_difference()

In [ ]:
time.get_frame

X both pass the critical distance-threshold and therefore our null hypothesis is rejected (not any significant differences between treatments)